In [ ]:
uf_status = {
 
}

with open("uf_config1.in", 'r') as f:
    linhas = f.readlines()
    for linha in linhas:
        partes = linha.strip().split()
        uf_status[partes[0]] = {partes[1]: partes[2]}
print(uf_status)



['int', '2', '1']
['mult', '2', '4']
['add', '1', '2']
['div', '1', '10']
{'int': {'2': '1'}, 'mult': {'2': '4'}, 'add': {'1': '2'}, 'div': {'1': '10'}}


In [ ]:
fu = {}
for uf, config in uf_status.items():
    for c in config:
        qnt = int(c)
        ciclos = int(config[c])
        fu[uf] = []
        for i in range(qnt):
            fu[uf].append({
                'id': f'{uf}{i+1}',
                'busy': False,
                'op': None,
                'fi': None,
                'fj': None,
                'fk': None,
                'qj': None,
                'qk': None,
                'rj': False,
                'rk': False,
                'ciclos_restantes': 0,
                'total_ciclos': ciclos
            })

print(fu)

{'int': [{'id': 'int1', 'busy': False, 'op': None, 'fi': None, 'fj': None, 'fk': None, 'qj': None, 'qk': None, 'rj': False, 'rk': False, 'ciclos_restantes': 0, 'total_ciclos': 1}, {'id': 'int2', 'busy': False, 'op': None, 'fi': None, 'fj': None, 'fk': None, 'qj': None, 'qk': None, 'rj': False, 'rk': False, 'ciclos_restantes': 0, 'total_ciclos': 1}], 'mult': [{'id': 'mult1', 'busy': False, 'op': None, 'fi': None, 'fj': None, 'fk': None, 'qj': None, 'qk': None, 'rj': False, 'rk': False, 'ciclos_restantes': 0, 'total_ciclos': 4}, {'id': 'mult2', 'busy': False, 'op': None, 'fi': None, 'fj': None, 'fk': None, 'qj': None, 'qk': None, 'rj': False, 'rk': False, 'ciclos_restantes': 0, 'total_ciclos': 4}], 'add': [{'id': 'add1', 'busy': False, 'op': None, 'fi': None, 'fj': None, 'fk': None, 'qj': None, 'qk': None, 'rj': False, 'rk': False, 'ciclos_restantes': 0, 'total_ciclos': 2}], 'div': [{'id': 'div1', 'busy': False, 'op': None, 'fi': None, 'fj': None, 'fk': None, 'qj': None, 'qk': None, 'rj'